# Week 1 – Financial Data Processing & Analysis

## Objective  
This notebook guides you through:  
- Loading historical stock data for 5 large-cap US companies  
- Cleaning and transforming the data  
- Calculating daily returns, moving averages, and volatility  
- Saving the final processed dataset including all columns and indicators  

---

## Step 1: Import Required Libraries
## Step 2: Load Historical Data from .txt Files


Each file is named like aapl.us.txt, msft.us.txt, etc.
Assumes files are uploaded to /content/ (Google Colab default)
## Step 3: Filter Data to Last 10 Years

In [11]:
import os
import pandas as pd
import numpy as np

# List of tickers to include
tickers = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL']

# Load and combine all data
dfs = []
for ticker in tickers:
    path = f"/content/{ticker.lower()}.us.txt"
    df = pd.read_csv(path, parse_dates=["Date"])
    df["Ticker"] = ticker.upper()
    dfs.append(df)

full_df = pd.concat(dfs)
full_df = full_df[full_df["Date"] >= "2007-11-10"]
full_df.set_index(["Ticker", "Date"], inplace=True)
full_df.sort_index(inplace=True)

## Step 4: Add Financial Indicators

Calculated columns:

- Daily Return
- 7-day Moving Average
- 30-day Moving Average
- 30-day Rolling Volatility (std deviation of returns)

## Step 5: Exploratory Analysis
1. Tesla had the highest average return over the 10-year period.
2. Tesla had the most volatile month in June 2010.

In [12]:
# 🔁 Add Financial Indicators
def add_indicators(df):
    df = df.sort_index(level=1)
    df["Daily Return"] = df.groupby(level=0)["Close"].pct_change()
    df["7D MA"] = df.groupby(level=0)["Close"].transform(lambda x: x.rolling(7).mean())
    df["30D MA"] = df.groupby(level=0)["Close"].transform(lambda x: x.rolling(30).mean())
    df["30D Volatility"] = df.groupby(level=0)["Daily Return"].transform(lambda x: x.rolling(30).std())
    return df

final_df = add_indicators(full_df)

# 📊 Exploratory Analysis
# 1. Which stock had the highest average return?
avg_returns = final_df.groupby(level=0)["Daily Return"].mean().sort_values(ascending=False)
print("Average Returns:\n", avg_returns)

# 2. Which stock had the most volatile month?
temp_df = final_df.copy()
temp_df["Month"] = temp_df.index.get_level_values(1).to_period("M")
monthly_vol = temp_df.groupby(["Ticker", "Month"])["Daily Return"].std()
most_volatile_month = monthly_vol.idxmax(), monthly_vol.max()
print("\nMost Volatile Month:\n", most_volatile_month)

Average Returns:
 Ticker
TSLA     0.002101
AMZN     0.001359
AAPL     0.001064
GOOGL    0.000645
MSFT     0.000589
Name: Daily Return, dtype: float64

Most Volatile Month:
 (('TSLA', Period('2010-06', 'M')), 0.28836212548779094)


In [14]:
final_df.to_csv("/content/processed_stock_data.csv")